In [ ]:
import pandas as pd
from sqlalchemy import text
from config.database import get_engine

In [ ]:
engine = get_engine()
print("Conexão estabelecida. Iniciando carga...")

# 1. Carga da Tabela Fato (Dados Silver)
df_fato = pd.read_csv('data/silver/dados_limpos.csv')
print(f"Lendo dados Fato (Silver): {df_fato.shape[0]} registros")

# AÇÃO: Carregar a FATO_SENTIMENTO
df_fato.to_sql('FATO_SENTIMENTO', engine, if_exists='replace', index=False)
print("✅ Tabela FATO_SENTIMENTO carregada.")

# 2. Carga da Tabela de Agregações (Dados Gold)
df_metricas = pd.read_csv('data/gold/metricas_diarias_sentimento.csv')
print(f"Lendo dados Métricas (Gold): {df_metricas.shape[0]} registros")

# AÇÃO: Carregar a METRICAS_DIARIAS
df_metricas.to_sql('METRICAS_DIARIAS', engine, if_exists='replace', index=False)
print("✅ Tabela METRICAS_DIARIAS carregada.")

# 3. Carga da Tabela Dimensão de Tempo (CRÍTICO)
# O Power BI precisa dessa tabela para criar o calendário.
# Ação: Gerar a DIM_TEMPO a partir das datas únicas da Fato
df_datas = df_fato[['data_postagem']].drop_duplicates()
df_datas['data'] = pd.to_datetime(df_datas['data_postagem']).dt.date

# Expansão da Dimensão de Tempo
df_datas['ano'] = df_datas['data'].apply(lambda x: x.year)
df_datas['mes'] = df_datas['data'].apply(lambda x: x.month)
df_datas['dia_da_semana'] = df_datas['data'].apply(lambda x: x.strftime('%A')) # Nome do dia da semana

# AÇÃO: Carregar a DIM_TEMPO
df_datas.to_sql('DIM_TEMPO', engine, if_exists='replace', index=False)
print("✅ Tabela DIM_TEMPO (Calendário) carregada.")

print("\n--- CARGA COMPLETA ---")